In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble  import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score, mean_squared_error, r2_score, accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn import tree

import sys
sys.path.insert(0, '../src/features/')
from custom_metric import *

pd.set_option("display.max.columns", None)

In [ ]:
# define noise levels

levels = [1, 2, 3]

## Random Forest 1 - Multiclass
#### Features = 'capacity', 'vib1', 'vib2', 'amp_uni', 'mic1', 'mic2'
#### Value = 'type_cat'

In [ ]:
drop_list = ['Unnamed: 0', 'datetime', 'date', 'time', 'pump', 'pump_cat', 'capacity_cat', 'anomaly', 'anomaly_cat', 'anomaly_binary',
       'anomaly_binary_cat', 'type', 'vib1', 'vib1_x', 'vib1_y',
       'vib1_z', 'vib2', 'vib2_x', 'vib2_y', 'vib2_z', 'amp1', 'amp2',
       'amp_uni', 'mic1', 'mic2',
       'vib1_x_noise', 'vib1_x_with_noise', 'vib1_y_noise',
       'vib1_y_with_noise', 'vib1_z_noise', 'vib1_z_with_noise', 'vib2_x_noise', 'vib2_x_with_noise', 'vib2_y_noise',
       'vib2_y_with_noise', 'vib2_z_noise', 'vib2_z_with_noise', 'amp1_noise',
       'amp1_with_noise', 'amp2_noise', 'amp2_with_noise', 'mic1_noise', 'mic2_noise']

value = 'type_cat'

### Firstly, let's just train a model previosly hypertuned for clean data and see how the metrics change with change in noise level.

In [ ]:
accuracies = {}
custom_metrics = {}

for level in levels:
    
    #read in data
    data = pd.read_csv('../data/noisy_data_'+str(level)+'.csv')
    
    #drop the columns we don't need
    df = data.drop(drop_list, axis=1)
    
    #define features and value
    X = df.drop(value, axis=1)
    y = df[value]
    
    #create train and test datasets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.20, random_state = 100)
    
    #train the model
    model = RandomForestClassifier(random_state = 100, bootstrap = 'False', min_samples_split=5, max_features=2, n_estimators = 35)
    model.fit(X_train, y_train)
    
    #calculate accuracy and custom metric
    
    cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=100)
    scores1 = cross_val_score(model, X, y, cv=cv, n_jobs=1)
    accuracies[level] = scores1
    
    scores2 = cross_val_score(model, X, y, cv=cv, n_jobs=1, scoring=make_scorer(cedric_metric_nonbinary, greater_is_better=True))
    custom_metrics[level] = scores2

In [ ]:
noise_levels = accuracies.keys()
mean_accuracies = [accuracies[n].mean() for n in levels]
# show range of scores using min and max values
lower_lim = [accuracies[n].min() for n in levels]
upper_lim = [accuracies[n].max() for n in levels]

plt.plot(noise_levels, mean_accuracies)
plt.fill_between(noise_levels, lower_lim, upper_lim, alpha=0.3, color='gray')

plt.title(f'Random forest (for {value}) accuracies for different noise levels', fontsize=16)
plt.xlabel('noise level', fontsize=14)
plt.ylabel('CV accuracy score (mean score / range)', fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

fig = plt.gcf()
fig.set_size_inches(12, 6)

#plt.show()
plt.savefig('random_forest_noise.jpeg')

In [ ]:
noise_levels = custom_metrics.keys()
mean_custom = [custom_metrics[n].mean() for n in levels]
# show range of scores using min and max values
lower_lim = [custom_metrics[n].min() for n in levels]
upper_lim = [custom_metrics[n].max() for n in levels]

plt.plot(noise_levels, mean_custom)
plt.fill_between(noise_levels, lower_lim, upper_lim, alpha=0.3, color='gray')

plt.title(f'Random forest (for {value}) custom metrics for different noise levels', fontsize=16)
plt.xlabel('noise level', fontsize=14)
plt.ylabel('CV custom metric score (mean score / range)', fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

fig = plt.gcf()
fig.set_size_inches(12, 6)

#plt.show()
#plt.savefig('adaboost_noise.jpeg')

In [ ]:
for level in levels:
    
    print('Noise level: '+str(level)+'\n')
    print('Accuracy: '+ str(mean_accuracies[level-1])+'\n')
    print('Custom metric: '+ str(mean_custom[level-1])+'\n')
    print('**************')

### Now, let's try hypertuning the models for each noise level separately.

In [ ]:
#Let's prepare the dataframes for all noise levels

for level in levels:
    
    #read in data
    globals()[f'data_{level}'] = pd.read_csv('../data/noisy_data_'+str(level)+'.csv')
    
    #drop the columns we don't need
    globals()[f'df_{level}'] = globals()[f'data_{level}'].drop(drop_list, axis=1)
    
    #define features and value
    globals()[f'X_{level}'] = globals()[f'df_{level}'].drop(value, axis=1)
    globals()[f'y_{level}'] = globals()[f'df_{level}'][value]
    
    #scale the data
    scaler = StandardScaler()  
    globals()[f'X_{level}'] = scaler.fit_transform(globals()[f'X_{level}']) 
    
    #create train and test datasets
    globals()[f'X_{level}_train'], globals()[f'X_{level}_val'], globals()[f'y_{level}_train'], globals()[f'y_{level}_val'] = train_test_split(X, y, test_size = 0.20, random_state = 100)
    

# Define parameters for hypertuning

n_estimators = [25, 35, 45]

max_features = [2,4,6]

min_samples_split = [10, 20, 40]

bootstrap = [True, False]

criterion = ['gini', 'entropy']

weights = [{1: 1000, 2: 1000, 3: 1000},
           {1: 3000, 2: 3000, 3: 3000},
           {1: 5000, 2: 5000, 3: 5000},
          {1: 10000, 2: 10000, 3: 10000}]

max_depth = [5, 10, 15]

# Create the grid
search_params = {'n_estimators': n_estimators,
                 'max_features': max_features,
                 'min_samples_split': min_samples_split,
                 'bootstrap': bootstrap,
                 'criterion': criterion,
                 'class_weight': weights,
                 'max_depth': max_depth}

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=100)


### Noise level 1

In [ ]:
#train the model
model_1 = RandomForestClassifier(random_state = 100)

model_1.fit(X_1_train, y_1_train)


#perform hypertuning
model_1_v2 = GridSearchCV(model_1, search_params, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_nonbinary, greater_is_better=True))

model_1_v2.fit(X_1_train, y_1_train)

best_params = model_1_v2.best_params_

print('Best Parameters', model_1_v2.best_params_)

In [ ]:
#train the hypetuned model

model_1_v3 = RandomForestClassifier(random_state = 100, bootstrap = 'True', min_samples_split=40, max_features=2, n_estimators = 35, class_weight={1: 5000, 2: 5000, 3: 5000}, criterion='entropy', max_depth=10)
model_1_v3.fit(X_1_train, y_1_train)

scores = cross_val_score(model_1_v3, X_1, y_1, cv=cv, n_jobs=-1)
print('Cross Validation accuracy scores: %s' % scores)
print('Cross Validation accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))
print('\n')

scores = cross_val_score(model_1_v3, X_1, y_1, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_nonbinary, greater_is_better=True))
print('Cross Validation custom metric scores: %s' % scores)
print('Cross Validation custom metric: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))


### Noise level 2

In [ ]:
#train the model
model_2 = RandomForestClassifier(random_state = 100)

model_2.fit(X_2_train, y_2_train)


#perform hypertuning
model_2_v2 = GridSearchCV(model_2, search_params, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_nonbinary, greater_is_better=True))

model_2_v2.fit(X_2_train, y_2_train)

best_params = model_2_v2.best_params_  

print('Best Parameters', model_2_v2.best_params_)

In [ ]:
#train the hypetuned model

model_2_v3 = RandomForestClassifier(random_state = 100, bootstrap = 'True', min_samples_split=40, max_features=2, n_estimators = 35, class_weight={1: 3000, 2: 3000, 3: 3000}, criterion='entropy', max_depth=10)
model_2_v3.fit(X_2_train, y_2_train)

scores = cross_val_score(model_2_v3, X_2, y_2, cv=cv, n_jobs=-1)
print('Cross Validation accuracy scores: %s' % scores)
print('Cross Validation accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))
print('\n')

scores = cross_val_score(model_2_v3, X_2, y_2, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_nonbinary, greater_is_better=True))
print('Cross Validation custom metric scores: %s' % scores)
print('Cross Validation custom metric: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

### Noise level 3

In [ ]:
#train the model
model_3 = RandomForestClassifier(random_state = 100)

model_3.fit(X_3_train, y_3_train)


#perform hypertuning
model_3_v2 = GridSearchCV(model_3, search_params, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_nonbinary, greater_is_better=True))

model_3_v2.fit(X_3_train, y_3_train)

best_params = model_3_v2.best_params_  

print('Best Parameters', model_3_v2.best_params_)

In [ ]:
#train the hypetuned model

model_3_v3 = RandomForestClassifier(random_state = 100, bootstrap = 'True', min_samples_split=40, max_features=2, n_estimators = 35, class_weight={1: 3000, 2: 3000, 3: 3000}, criterion='entropy', max_depth=10)
model_3_v3.fit(X_3_train, y_3_train)

scores = cross_val_score(model_3_v3, X_3, y_3, cv=cv, n_jobs=-1)
print('Cross Validation accuracy scores: %s' % scores)
print('Cross Validation accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))
print('\n')

scores = cross_val_score(model_3_v3, X_3, y_3, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_nonbinary, greater_is_better=True))
print('Cross Validation custom metric scores: %s' % scores)
print('Cross Validation custom metric: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

## Random forests 2 - Binary
#### Features = 'capacity', 'vib1', 'vib2', 'amp_uni', 'mic1', 'mic2'
#### Value = 'anomaly_binary'

In [ ]:
drop_list_b = ['Unnamed: 0', 'datetime', 'date', 'time', 'pump', 'pump_cat', 'capacity_cat', 'anomaly', 'anomaly_cat', 'type_cat',
       'anomaly_binary_cat', 'type', 'vib1', 'vib1_x', 'vib1_y',
       'vib1_z', 'vib2', 'vib2_x', 'vib2_y', 'vib2_z', 'amp1', 'amp2',
       'amp_uni', 'mic1', 'mic2',
       'vib1_x_noise', 'vib1_x_with_noise', 'vib1_y_noise',
       'vib1_y_with_noise', 'vib1_z_noise', 'vib1_z_with_noise', 'vib2_x_noise', 'vib2_x_with_noise', 'vib2_y_noise',
       'vib2_y_with_noise', 'vib2_z_noise', 'vib2_z_with_noise', 'amp1_noise',
       'amp1_with_noise', 'amp2_noise', 'amp2_with_noise', 'mic1_noise', 'mic2_noise']

value_b = 'anomaly_binary'

### Firstly, let's just train a model previosly hypertuned for clean data and see how the metrics change with change in noise level.

In [ ]:
accuracies = {}
custom_metrics = {}

for level in levels:
    
    #read in data
    data = pd.read_csv('../data/noisy_data_'+str(level)+'.csv')
    
    #drop the columns we don't need
    df = data.drop(drop_list_b, axis=1)
    
    #define features and value
    X = df.drop(value_b, axis=1)
    y = df[value_b]
    
    #create train and test datasets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.20, random_state = 100)
    
    #train the model
    model = RandomForestClassifier(random_state = 100, bootstrap = 'False', min_samples_split=5, max_features=2, n_estimators = 35)
    model.fit(X_train, y_train)
    
    #calculate accuracy and custom metric
    
    cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=100)
    scores1 = cross_val_score(model, X, y, cv=cv, n_jobs=1)
    accuracies[level] = scores1
    
    scores2 = cross_val_score(model, X, y, cv=cv, n_jobs=1, scoring=make_scorer(cedric_metric_binary, greater_is_better=True))
    custom_metrics[level] = scores2

In [ ]:
noise_levels = accuracies.keys()
mean_accuracies = [accuracies[n].mean() for n in levels]
# show range of scores using min and max values
lower_lim = [accuracies[n].min() for n in levels]
upper_lim = [accuracies[n].max() for n in levels]

plt.plot(noise_levels, mean_accuracies)
plt.fill_between(noise_levels, lower_lim, upper_lim, alpha=0.3, color='gray')

plt.title(f'Random forest (for {value_b}) accuracies for different noise levels', fontsize=16)
plt.xlabel('noise level', fontsize=14)
plt.ylabel('CV accuracy score (mean score / range)', fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

fig = plt.gcf()
fig.set_size_inches(12, 6)

#plt.show()
#plt.savefig('random_forest_noise.jpeg')

In [ ]:
noise_levels = custom_metrics.keys()
mean_custom = [custom_metrics[n].mean() for n in levels]
# show range of scores using min and max values
lower_lim = [custom_metrics[n].min() for n in levels]
upper_lim = [custom_metrics[n].max() for n in levels]

plt.plot(noise_levels, mean_custom)
plt.fill_between(noise_levels, lower_lim, upper_lim, alpha=0.3, color='gray')

plt.title(f'Random forest (for {value_b}) custom metrics for different noise levels', fontsize=16)
plt.xlabel('noise level', fontsize=14)
plt.ylabel('CV custom metric score (mean score / range)', fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

fig = plt.gcf()
fig.set_size_inches(12, 6)

#plt.show()
#plt.savefig('adaboost_noise.jpeg')

In [ ]:
for level in levels:
    
    print('Noise level: '+str(level)+'\n')
    print('Accuracy: '+ str(mean_accuracies[level-1])+'\n')
    print('Custom metric: '+ str(mean_custom[level-1])+'\n')
    print('**************')

### Now, let's try hypertuning the models for each noise level separately.


In [ ]:
#Let's prepare the dataframes for all noise levels

for level in levels:
    
    #drop the columns we don't need
    globals()[f'df_{level}_b'] = globals()[f'data_{level}'].drop(drop_list_b, axis=1)
    
    #define features and value
    globals()[f'X_{level}_b'] = globals()[f'df_{level}_b'].drop(value_b, axis=1)
    globals()[f'y_{level}_b'] = globals()[f'df_{level}_b'][value_b]
    
    #scale the data
    scaler = StandardScaler()  
    globals()[f'X_{level}_b'] = scaler.fit_transform(globals()[f'X_{level}_b']) 
    
    #create train and test datasets
    globals()[f'X_{level}_b_train'], globals()[f'X_{level}_b_val'], globals()[f'y_{level}_b_train'], globals()[f'y_{level}_b_val'] = train_test_split(globals()[f'X_{level}'], globals()[f'y_{level}'], test_size = 0.20, random_state = 100)
    

# Define parameters for hypertuning

n_estimators = [25, 35, 45]

max_features = [2,4,6]

min_samples_split = [10, 20, 40]

bootstrap = [True, False]

criterion = ['gini', 'entropy']

weights = [{1: 3000}, {1:5000}, {1:10000}]

max_depth = [5, 10]

# Create the grid
search_params = {'n_estimators': n_estimators,
                 'max_features': max_features,
                 'min_samples_split': min_samples_split,
                 'bootstrap': bootstrap,
                 'criterion': criterion,
                 'class_weight': weights,
                 'max_depth': max_depth}

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=100)


### Noise level 1

In [ ]:
#train the model
model_1_b = RandomForestClassifier(random_state = 100)

model_1_b.fit(X_1_b_train, y_1_b_train)


#perform hypertuning
model_1_b_v2 = GridSearchCV(model_1_b, search_params, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_binary, greater_is_better=True))

model_1_b_v2.fit(X_1_b_train, y_1_b_train)

best_params = model_1_b_v2.best_params_  

print('Best Parameters', model_1_b_v2.best_params_)

In [ ]:
#train the hypetuned model

model_1_b_v3 = RandomForestClassifier(random_state = 100, bootstrap = 'True', min_samples_split=40, max_features=2, n_estimators = 35, class_weight={1: 3000}, criterion='entropy', max_depth=10)
model_1_b_v3.fit(X_1_b_train, y_1_b_train)

scores = cross_val_score(model_1_b_v3, X_1_b, y_1_b, cv=cv, n_jobs=-1)
print('Cross Validation accuracy scores: %s' % scores)
print('Cross Validation accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))
print('\n')

scores = cross_val_score(model_1_b_v3, X_1_b, y_1_b, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_binary, greater_is_better=True))
print('Cross Validation custom metric scores: %s' % scores)
print('Cross Validation custom metric: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

### Noise level 2

In [ ]:
#train the model
model_2_b = RandomForestClassifier(random_state = 100)

model_2_b.fit(X_2_b_train, y_2_b_train)


#perform hypertuning
model_2_b_v2 = GridSearchCV(model_2_b, search_params, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_binary, greater_is_better=True))

model_2_b_v2.fit(X_2_b_train, y_2_b_train)

best_params = model_2_b_v2.best_params_  

# print('Best Parameters', model_1_2_v2.best_params_) <- there's a bug here, do not want to rerun the hypertuning

In [ ]:
print('Best Parameters', model_2_b_v2.best_params_)

In [ ]:
#train the hypetuned model

model_2_b_v3 = RandomForestClassifier(random_state = 100, bootstrap = 'True', min_samples_split=40, max_features=2, n_estimators = 35, class_weight={1: 10000}, criterion='entropy', max_depth=10)
model_2_b_v3.fit(X_2_b_train, y_2_b_train)

scores = cross_val_score(model_2_b_v3, X_2_b, y_2_b, cv=cv, n_jobs=-1)
print('Cross Validation accuracy scores: %s' % scores)
print('Cross Validation accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))
print('\n')

scores = cross_val_score(model_2_b_v3, X_2_b, y_2_b, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_binary, greater_is_better=True))
print('Cross Validation custom metric scores: %s' % scores)
print('Cross Validation custom metric: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

### Noise level 3

In [ ]:
#train the model
model_3_b = RandomForestClassifier(random_state = 100)

model_3_b.fit(X_3_b_train, y_3_b_train)


#perform hypertuning
model_3_b_v2 = GridSearchCV(model_3_b, search_params, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_binary, greater_is_better=True))

model_3_b_v2.fit(X_3_b_train, y_3_b_train)

best_params = model_3_b_v2.best_params_  

print('Best Parameters', model_3_b_v2.best_params_)

In [ ]:
#train the hypetuned model

model_3_b_v3 = RandomForestClassifier(random_state = 100, bootstrap = 'True', min_samples_split=40, max_features=2, n_estimators = 35, class_weight={1: 10000}, criterion='entropy', max_depth=10)
model_3_b_v3.fit(X_3_b_train, y_3_b_train)

scores = cross_val_score(model_3_b_v3, X_3_b, y_3_b, cv=cv, n_jobs=-1)
print('Cross Validation accuracy scores: %s' % scores)
print('Cross Validation accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))
print('\n')

scores = cross_val_score(model_3_b_v3, X_3_b, y_3_b, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_binary, greater_is_better=True))
print('Cross Validation custom metric scores: %s' % scores)
print('Cross Validation custom metric: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

## Random Forest 3 - Multiclass with original features
#### Features = 'capacity', 'vib1_x', 'vib1_y', 'vib1_z', 'vib2_x', 'vib2_y', 'vib2_z','amp_uni', 'mic1', 'mic2'
#### Value = 'type_cat'

In [ ]:
drop_list = ['Unnamed: 0', 'datetime', 'date', 'time', 'pump', 'pump_cat', 'capacity_cat', 'anomaly', 'anomaly_cat', 'anomaly_binary',
       'anomaly_binary_cat', 'type', 'vib1', 'vib1_x', 'vib1_y',
       'vib1_z', 'vib2', 'vib2_x', 'vib2_y', 'vib2_z', 'amp1', 'amp2',
       'amp_uni', 'mic1', 'mic2',
       'vib1_x_noise', 'vib1_with_noise', 'vib1_y_noise','vib1_z_noise', 'vib2_x_noise', 'vib2_with_noise', 'vib2_y_noise', 'vib2_z_noise', 'amp1_noise',
       'amp_uni_with_noise', 'amp2_noise', 'mic1_noise', 'mic2_noise']

value = 'type_cat'

In [ ]:
#Let's prepare the dataframes for all noise levels

for level in levels:
    
    #read in data
    globals()[f'data_{level}'] = pd.read_csv('../data/noisy_data_'+str(level)+'.csv')
    
    #drop the columns we don't need
    globals()[f'df_{level}'] = globals()[f'data_{level}'].drop(drop_list, axis=1)
    
    #define features and value
    globals()[f'X_{level}'] = globals()[f'df_{level}'].drop(value, axis=1)
    globals()[f'y_{level}'] = globals()[f'df_{level}'][value]
    
    #scale the data
    scaler = StandardScaler()  
    globals()[f'X_{level}'] = scaler.fit_transform(globals()[f'X_{level}']) 
    
    #create train and test datasets
    globals()[f'X_{level}_train'], globals()[f'X_{level}_val'], globals()[f'y_{level}_train'], globals()[f'y_{level}_val'] = train_test_split(X, y, test_size = 0.20, random_state = 100)
    

# Define parameters for hypertuning

n_estimators = [25, 35, 45]

max_features = [2,4,6]

min_samples_split = [10, 20, 40]

bootstrap = [True, False]

criterion = ['gini', 'entropy']

weights = [{1: 1000, 2: 1000, 3: 1000},
           {1: 3000, 2: 3000, 3: 3000},
           {1: 5000, 2: 5000, 3: 5000},
          {1: 10000, 2: 10000, 3: 10000}]

max_depth = [5, 10]

# Create the random grid
search_params = {'n_estimators': n_estimators,
                 'max_features': max_features,
                 'min_samples_split': min_samples_split,
                 'bootstrap': bootstrap,
                 'criterion': criterion,
                 'class_weight': weights,
                 'max_depth': max_depth}

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=100)


### Noise level 1

In [ ]:
#train the model
model_1 = RandomForestClassifier(random_state = 100)

model_1.fit(X_1_train, y_1_train)


#perform hypertuning
model_1_v2 = GridSearchCV(model_1, search_params, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_nonbinary, greater_is_better=True))

model_1_v2.fit(X_1_train, y_1_train)

best_params = model_1_v2.best_params_

print('Best Parameters', model_1_v2.best_params_)

In [ ]:
#train the hypetuned model

model_1_v3 = RandomForestClassifier(random_state = 100, bootstrap = 'True', min_samples_split=40, max_features=2, n_estimators = 35, class_weight={1: 5000, 2: 5000, 3: 5000}, criterion='entropy', max_depth=10)
model_1_v3.fit(X_1_train, y_1_train)

scores = cross_val_score(model_1_v3, X_1, y_1, cv=cv, n_jobs=-1)
print('Cross Validation accuracy scores: %s' % scores)
print('Cross Validation accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))
print('\n')

scores = cross_val_score(model_1_v3, X_1, y_1, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_nonbinary, greater_is_better=True))
print('Cross Validation custom metric scores: %s' % scores)
print('Cross Validation custom metric: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))


### Noise level 2

In [ ]:
#train the model
model_2 = RandomForestClassifier(random_state = 100)

model_2.fit(X_2_train, y_2_train)


#perform hypertuning
model_2_v2 = GridSearchCV(model_2, search_params, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_nonbinary, greater_is_better=True))

model_2_v2.fit(X_2_train, y_2_train)

best_params = model_2_v2.best_params_

print('Best Parameters', model_2_v2.best_params_)

In [ ]:
#train the hypetuned model

model_2_v3 = RandomForestClassifier(random_state = 100, bootstrap = 'True', min_samples_split=40, max_features=2, n_estimators = 35, class_weight={1: 5000, 2: 5000, 3: 5000}, criterion='entropy', max_depth=10)
model_2_v3.fit(X_2_train, y_2_train)

scores = cross_val_score(model_2_v3, X_2, y_2, cv=cv, n_jobs=-1)
print('Cross Validation accuracy scores: %s' % scores)
print('Cross Validation accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))
print('\n')

scores = cross_val_score(model_2_v3, X_2, y_2, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_nonbinary, greater_is_better=True))
print('Cross Validation custom metric scores: %s' % scores)
print('Cross Validation custom metric: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

### Noise level 3

In [ ]:
#train the model
model_3 = RandomForestClassifier(random_state = 100)

model_3.fit(X_3_train, y_3_train)


#perform hypertuning
model_3_v2 = GridSearchCV(model_3, search_params, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_nonbinary, greater_is_better=True))

model_3_v2.fit(X_3_train, y_3_train)

best_params = model_3_v2.best_params_

print('Best Parameters', model_3_v2.best_params_)

In [ ]:
#train the hypetuned model

model_3_v3 = RandomForestClassifier(random_state = 100, bootstrap = 'True', min_samples_split=40, max_features=2, n_estimators = 35, class_weight={1: 5000, 2: 5000, 3: 5000}, criterion='entropy', max_depth=10)
model_3_v3.fit(X_3_train, y_3_train)

scores = cross_val_score(model_3_v3, X_3, y_3, cv=cv, n_jobs=-1)
print('Cross Validation accuracy scores: %s' % scores)
print('Cross Validation accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))
print('\n')

scores = cross_val_score(model_3_v3, X_3, y_3, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_nonbinary, greater_is_better=True))
print('Cross Validation custom metric scores: %s' % scores)
print('Cross Validation custom metric: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

## Random Forest 4 - Binary with original features
#### Features = 'capacity', 'vib1_x', 'vib1_y', 'vib1_z', 'vib2_x', 'vib2_y', 'vib2_z','amp_uni', 'mic1', 'mic2'
#### Value = 'anomaly_binary'

In [ ]:
drop_list_b = ['Unnamed: 0', 'datetime', 'date', 'time', 'pump', 'pump_cat', 'capacity_cat', 'anomaly', 'anomaly_cat', 'type_cat',
       'anomaly_binary_cat', 'type', 'vib1', 'vib1_x', 'vib1_y',
       'vib1_z', 'vib2', 'vib2_x', 'vib2_y', 'vib2_z', 'amp1', 'amp2',
       'amp_uni', 'mic1', 'mic2',
       'vib1_x_noise', 'vib1_with_noise', 'vib1_y_noise','vib1_z_noise', 'vib2_x_noise', 'vib2_with_noise', 'vib2_y_noise', 'vib2_z_noise', 'amp1_noise',
       'amp_uni_with_noise', 'amp2_noise', 'mic1_noise', 'mic2_noise']

value_b = 'anomaly_binary'

In [ ]:
#Let's prepare the dataframes for all noise levels

for level in levels:
    
    #drop the columns we don't need
    globals()[f'df_{level}_b'] = globals()[f'data_{level}'].drop(drop_list_b, axis=1)
    
    #define features and value
    globals()[f'X_{level}_b'] = globals()[f'df_{level}_b'].drop(value_b, axis=1)
    globals()[f'y_{level}_b'] = globals()[f'df_{level}_b'][value_b]
    
    #scale the data
    scaler = StandardScaler()  
    globals()[f'X_{level}_b'] = scaler.fit_transform(globals()[f'X_{level}_b']) 
    
    #create train and test datasets
    globals()[f'X_{level}_b_train'], globals()[f'X_{level}_b_val'], globals()[f'y_{level}_b_train'], globals()[f'y_{level}_b_val'] = train_test_split(globals()[f'X_{level}_b'], globals()[f'y_{level}_b'], test_size = 0.20, random_state = 100)
    

# Define parameters for hypertuning

n_estimators_b = [25, 35, 45]

max_features_b = [2,4,6]

min_samples_split_b = [10, 20, 40]

bootstrap_b = [True, False]

criterion_b = ['gini', 'entropy']

weights_b = [{1: 1000},
           {1: 3000},
           {1: 5000},
          {1: 10000}]

max_depth_b = [5, 10]

# Create the random grid
search_params_b = {'n_estimators': n_estimators_b,
                 'max_features': max_features_b,
                 'min_samples_split': min_samples_split_b,
                 'bootstrap': bootstrap_b,
                 'criterion': criterion_b,
                 'class_weight': weights_b,
                 'max_depth': max_depth_b}

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=100)


### Noise level 1

In [ ]:
#train the model
model_1_b = RandomForestClassifier(random_state = 100)

model_1_b.fit(X_1_b_train, y_1_b_train)


#perform hypertuning
model_1_b_v2 = GridSearchCV(model_1_b, search_params_b, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_binary, greater_is_better=True))

model_1_b_v2.fit(X_1_b_train, y_1_b_train)

best_params = model_1_b_v2.best_params_  

print('Best Parameters', model_1_b_v2.best_params_)

In [ ]:
#train the hypetuned model

model_1_b_v3 = RandomForestClassifier(random_state = 100, bootstrap = 'False', min_samples_split=40, max_features=2, n_estimators = 45, class_weight={1: 5000}, criterion='entropy', max_depth=10)
model_1_b_v3.fit(X_1_b_train, y_1_b_train)

scores = cross_val_score(model_1_b_v3, X_1_b, y_1_b, cv=cv, n_jobs=-1)
print('Cross Validation accuracy scores: %s' % scores)
print('Cross Validation accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))
print('\n')

scores = cross_val_score(model_1_b_v3, X_1_b, y_1_b, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_binary, greater_is_better=True))
print('Cross Validation custom metric scores: %s' % scores)
print('Cross Validation custom metric: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

### Noise level 2

In [ ]:
#train the model
model_2_b = RandomForestClassifier(random_state = 100)

model_2_b.fit(X_2_b_train, y_2_b_train)


#perform hypertuning
model_2_b_v2 = GridSearchCV(model_2_b, search_params_b, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_binary, greater_is_better=True))

model_2_b_v2.fit(X_2_b_train, y_2_b_train)

best_params = model_2_b_v2.best_params_  

print('Best Parameters', model_2_b_v2.best_params_)

In [ ]:
#train the hypetuned model

#model_2_b_v3 = RandomForestClassifier(random_state = 100, bootstrap = 'True', min_samples_split=20, max_features=2, n_estimators = 45, class_weight={1: 1000}, criterion='entropy', max_depth=10)
model_2_b_v3 = RandomForestClassifier(random_state = 100, bootstrap = 'False', min_samples_split=40, max_features=2, n_estimators = 45, class_weight={1: 5000}, criterion='entropy', max_depth=10)
model_2_b_v3.fit(X_2_b_train, y_2_b_train)

scores = cross_val_score(model_2_b_v3, X_2_b, y_2_b, cv=cv, n_jobs=-1)
print('Cross Validation accuracy scores: %s' % scores)
print('Cross Validation accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))
print('\n')

scores = cross_val_score(model_2_b_v3, X_2_b, y_2_b, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_binary, greater_is_better=True))
print('Cross Validation custom metric scores: %s' % scores)
print('Cross Validation custom metric: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

### Noise level 3

In [ ]:
#train the model
model_3_b = RandomForestClassifier(random_state = 100)

model_3_b.fit(X_3_b_train, y_3_b_train)


#perform hypertuning
model_3_b_v2 = GridSearchCV(model_3_b, search_params_b, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_binary, greater_is_better=True))

model_3_b_v2.fit(X_3_b_train, y_3_b_train)

best_params = model_3_b_v2.best_params_  

print('Best Parameters', model_3_b_v2.best_params_)

In [ ]:
#train the hypetuned model

#model_3_b_v3 = RandomForestClassifier(random_state = 100, bootstrap = 'True', min_samples_split=40, max_features=2, n_estimators = 35, class_weight={1: 3000}, criterion='entropy', max_depth=10)
model_3_b_v3 = RandomForestClassifier(random_state = 100, bootstrap = 'False', min_samples_split=40, max_features=2, n_estimators = 45, class_weight={1: 5000}, criterion='entropy', max_depth=10)

model_3_b_v3.fit(X_3_b_train, y_3_b_train)

scores = cross_val_score(model_3_b_v3, X_3_b, y_3_b, cv=cv, n_jobs=-1)
print('Cross Validation accuracy scores: %s' % scores)
print('Cross Validation accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))
print('\n')

scores = cross_val_score(model_3_b_v3, X_3_b, y_3_b, cv=cv, n_jobs=-1, scoring=make_scorer(cedric_metric_binary, greater_is_better=True))
print('Cross Validation custom metric scores: %s' % scores)
print('Cross Validation custom metric: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

## In case of noise levels 2 and 3 parameters hypertuned for noise level 1 give better results than parameters hypertuned for noise levels 2 and 3 accordingly.